In [9]:
from pulp import LpProblem, LpVariable, lpSum, LpBinary, PULP_CBC_CMD, LpStatus
import pandas as pd

# Assuming `indiana_population_cleaned` is already loaded and cleaned as previously shown
indiana_population_cleaned = pd.read_csv('/Users/jck/Documents/MSDS 460/Module 6/Assignment3/Indiana Population Dataset.csv')
# For adjacency, we'll need a data structure like this, assuming it's available:
# adjacency = {'County1': ['County2', 'County3'], ...}

# Define the number of districts
num_districts = 9

# Create the model
model = LpProblem("Indiana_Redistricting", LpBinary)

# Decision variables: x[county][district] == 1 if county is in district, 0 otherwise
x = LpVariable.dicts("county_district", (indiana_population_cleaned['Geography'], range(num_districts)), cat='Binary')

# Objective: None specified, could be minimizing the standard deviation of population across districts
# This would ensure compactness by minimizing the population spread

# Constraints

# Each county must be in exactly one district
for county in indiana_population_cleaned['Geography']:
    model += lpSum(x[county][d] for d in range(num_districts)) == 1, f"one_district_per_county_{county}"

# Population balance across districts
avg_population = indiana_population_cleaned['Total'].sum() / num_districts
for d in range(num_districts):
    model += lpSum(indiana_population_cleaned.loc[indiana_population_cleaned['Geography'] == county, 'Total'].item() * x[county][d] for county in indiana_population_cleaned['Geography']) \
             <= avg_population * 1.05, f"max_population_district_{d}"
    model += lpSum(indiana_population_cleaned.loc[indiana_population_cleaned['Geography'] == county, 'Total'].item() * x[county][d] for county in indiana_population_cleaned['Geography']) \
             >= avg_population * 0.95, f"min_population_district_{d}"

# Adjacency constraint (example for a single district, should be expanded)
# model += x['County1'][0] + x['County2'][0] <= 1, "adjacency_example"

# Solve the problem
solver = PULP_CBC_CMD(msg=True)
model.solve(solver)

# Output results
print("Status:", LpStatus[model.status])
for v in model.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)

# Check outputs and interpret results


KeyError: 'Geography'